In [23]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-02-15 14:49:39--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: 'input.txt.1'

input.txt.1         100%[===================>]   1.06M  6.52MB/s    in 0.2s    

2023-02-15 14:49:40 (6.52 MB/s) - 'input.txt.1' saved [1115394/1115394]



In [24]:
#read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [25]:
print('lenght of the dataset in characters: ', len(text))

lenght of the dataset in characters:  1115394


In [26]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [27]:
#here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print('vocab size:',vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size: 65


In [28]:
#create a mapping from charaters to integers 
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder : take a string, output a list of integers
decode = lambda l: ''.join([itos[i]  for i in l]) #decoder: take a list  of int's and  output a string

encoding = encode("hii there")
print(encoding)
print(decode(encoding))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [29]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use Pytorch 
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) #the 1000 characters we looked at earlier will to the GPT look like this

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [30]:
# Splitting the dataset into train and validation sets
n = int(0.9 * len(data)) # firtst 90% train data, rest 10%val
train_data = data[:n]
val_data = data[n:]

In [31]:
block_size = 8
arr = train_data[:block_size+1]
arr.shape
arr,arr[:4],arr[4:]

(tensor([18, 47, 56, 57, 58,  1, 15, 47, 58]),
 tensor([18, 47, 56, 57]),
 tensor([58,  1, 15, 47, 58]))

In [32]:
x = train_data[:block_size]
y = train_data[1:block_size+1] 
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [33]:
torch.manual_seed(1337)
batch_size = 4 #how many independent sequences will we process in parallel?
block_size = 8 #what is the maximum context lenght for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data 
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('-----')

for b in range(batch_size): #batch dimension
    for t in range(block_size): #time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f" when input is  {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-----
 when input is  [24] the target: 43
 when input is  [24, 43] the target: 58
 when input is  [24, 43, 58] the target: 5
 when input is  [24, 43, 58, 5] the target: 57
 when input is  [24, 43, 58, 5, 57] the target: 1
 when input is  [24, 43, 58, 5, 57, 1] the target: 46
 when input is  [24, 43, 58, 5, 57, 1, 46] the target: 43
 when input is  [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
 when input is  [44] the target: 53
 when input is  [44, 53] the target: 56
 when input is  [44, 53, 56] the target: 1
 when input is  [44, 53, 56, 1] the target: 58
 when input is  [44, 53, 56, 1, 58] the targe

In [34]:
print(xb) #our input to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [35]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        #each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        #idx and targets are both(B, T) tensor of integers
        logits = self.token_embedding_table(idx) #(B, T, C) - batch, time, channel
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # making this re-arrangement so that the array conforms to what pytthorch expects
                                        # for the cross-entropy loss
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits,loss 
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last timestep
            logits = logits[:, -1, :] #Becomes (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B,C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            # append sampled index to the runnning sequence 
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [36]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb) 
print(logits.shape)
print(loss)

idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [37]:
#create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [38]:
batch_size = 32
for steps in range(10000):

    #sample a batch of data
    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


2.382369041442871


In [39]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=400)[0].tolist()))


lso br. ave aviasurf my, yxMPZI ivee iuedrd whar ksth y h bora s be hese, woweee; the! KI 'de, ulseecherd d o blllando;LUCEO, oraingofof win!
RIfans picspeserer hee tha,
TOFonk? me ain ckntoty ded. bo'llll st ta d:
ELIS me hurf lal y, ma dus pe athouo
BEY:! Indy; by s afreanoo adicererupa anse tecorro llaus a!
OLeneerithesinthengove fal amas trr
TI ar I t, mes, n IUSt my w, fredeeyove
THek' merer,


The mathematical trick in self-attention

In [40]:
# conisder the following toy example
torch.manual_seed(1337)
B, T, C = 4, 8, 2 # batch, time, channels
# x = torch.randn(B,T,C)
x = torch.arange(B*T*C,dtype=torch.float32)
x = x.view(B,T,C)
x.shape
print(x)

tensor([[[ 0.,  1.],
         [ 2.,  3.],
         [ 4.,  5.],
         [ 6.,  7.],
         [ 8.,  9.],
         [10., 11.],
         [12., 13.],
         [14., 15.]],

        [[16., 17.],
         [18., 19.],
         [20., 21.],
         [22., 23.],
         [24., 25.],
         [26., 27.],
         [28., 29.],
         [30., 31.]],

        [[32., 33.],
         [34., 35.],
         [36., 37.],
         [38., 39.],
         [40., 41.],
         [42., 43.],
         [44., 45.],
         [46., 47.]],

        [[48., 49.],
         [50., 51.],
         [52., 53.],
         [54., 55.],
         [56., 57.],
         [58., 59.],
         [60., 61.],
         [62., 63.]]])


In [41]:
# We want x[b, t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C),dtype=torch.float32)
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [42]:
# version 2
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
wei
xbow2 = wei @ x # (B{pythorch adds this extra batch},T, T) @ (B, T, C)   -> (B,T,C)
                # it multiplies for each batch element T,T @ T,C
torch.allclose(xbow, xbow2)

True

In [43]:
# version 3: use softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow,xbow3)

True

In [ ]:
# how self attention works:
# each token/node will emit two vectors 
# a query and a key #query - what am i looking for # key - what do i contain
# the way we get affinities between tokens in a sequence is
# we basically just do a Dot product between the keys and the queries 
# so my query dot products with all the keys of all the other tokens
# and the dot product becomes the weight (wei)
# and if the key and the query are aligned they will interact to a very high amount
# and then i will get to learn more about that specific token 
# as opposed to any other token in the equence 

In [80]:
# version 4 self-attention!
torch.manual_seed(1337)
B, T, C = 4, 8, 32 # batch time channels
x = torch.randn(B, T, C)

# let's see a single Head perform  self-attention
head_size = 16
key  = nn.Linear(C, head_size, bias=False) # bias is false so these are going just to apply matrix multiply with some fixed weights
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, head_size=16)
q = query(x) # (B, T<, head_size=16)
# wei -  the affinities between these 
wei = q @ k.transpose(-2,-1) # (B, T, 16) @ (B, 16, T) -> (B, T, T)


tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf')) #deleting this line will allow all nodes to talk to each other
                                                #useful when u do sentiment analysis
wei = F.softmax(wei, dim=-1) 

v = value(x) 
out = wei @ v
# out = wei @ x
out.shape



torch.Size([4, 8, 16])

In [81]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [119]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2,-1) * head_size**(-0.5) # the scaling is used to control the variance at initialization

In [120]:
k.var()

tensor(0.9279)

In [121]:
q.var()

tensor(0.9473)

In [122]:
wei.var()

tensor(0.9156)

In [123]:
t1 = torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])
torch.softmax(t1, dim=0)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [124]:
t2 = t1 *8
t2

tensor([ 0.8000, -1.6000,  2.4000, -1.6000,  4.0000])

In [125]:
torch.softmax(t2, dim=-1) # having larger values makes the softmax to tend to one of them 

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])